In [121]:
import torch
import shutil
import os
import cv2
import numpy as np
import yaml
import re
from PIL import Image
from io import BytesIO
import torchvision.transforms as tf
import io

In [31]:
cam = "02"
first_frame = 0
last_frame = 99
nb_frames = last_frame - first_frame + 1
target_im_folder = f"test_images_{cam}_{first_frame}_{last_frame}"

source_folder = f"/home/angelika/pixelsplat_fisheye/datasets/kitti360/2013_05_28_drive_0000_sync/image_{cam}/data_rgb"
target_folder = f"/home/angelika/pixelsplat_fisheye/datasets/kitti360/{target_im_folder}"

if not os.path.exists(target_folder):
    os.makedirs(target_folder)


# Get the list of filenames in the source folder
filenames = sorted(os.listdir(source_folder))
available_frames = []

# Iterate over the filenames and copy images to the destination folder
for i, filename in enumerate(filenames):
    if i < nb_frames:
        available_frames.append(i)
        source_path = os.path.join(source_folder, filename)
        filename_dest = filename
        destination_path = os.path.join(target_folder, filename_dest)
        
        # shutil.copy2(source_path, destination_path)


In [36]:
# Copy camera poses
pose_file = f"/home/angelika/pixelsplat_fisheye/datasets/kitti360/2013_05_28_drive_0000_sync/all_cam{cam[1:]}_to_world.txt"

# Extract poses
cam2world = []

# Read cam2world matrices
for line in open(pose_file, 'r').readlines():
    value = list(map(float, line.strip().split(" ")))
    cam2world.append(np.array(value[1:]))

cam2world = np.array(cam2world)
cam2world = cam2world[first_frame:last_frame+1]

# Save only the poses to a .txt file
with open(f'{target_folder}/camera_poses.txt', 'w') as file:
    for i, row in enumerate(cam2world):
        # line = [filenames[available_frames[i]]] + [str(x) for x in row]
        line = [str(available_frames[i])] + [str(x) for x in row]
        line = ' '.join(line) + '\n'
        # file.write(line)

In [103]:
image_dict = {}

image_dict['url'] = ""
image_dict['key'] = f"kitti360_cam{cam}_{first_frame}_{last_frame}"

timestamps = torch.as_tensor(available_frames)
image_dict['timestamps'] = timestamps

In [104]:
cam2world12 = torch.tensor(cam2world[:, :12])
two_zeros = torch.zeros((nb_frames, 2))
intrinsics = torch.zeros((nb_frames, 4))


In [105]:
def readYAMLFile(fileName):
    '''make OpenCV YAML file compatible with python'''
    ret = {}
    skip_lines=1    # Skip the first line which says "%YAML:1.0". Or replace it with "%YAML 1.0"
    with open(fileName) as fin:
        for i in range(skip_lines):
            fin.readline()
        yamlFileOut = fin.read()
        myRe = re.compile(r":([^ ])")   # Add space after ":", if it doesn't exist. Python yaml requirement
        yamlFileOut = myRe.sub(r': \1', yamlFileOut)
        ret = yaml.safe_load(yamlFileOut)
    return ret

intrinsics_file = f'/home/angelika/datasets/kitti_360/calibration/image_{cam}.yaml'
intrinsics_dict = readYAMLFile(intrinsics_file)

intrinsics[:, 0] = torch.tensor(intrinsics_dict['projection_parameters']['gamma1']).repeat(nb_frames)
intrinsics[:, 1] = torch.tensor(intrinsics_dict['projection_parameters']['gamma2']).repeat(nb_frames)
intrinsics[:, 2] = torch.tensor(intrinsics_dict['projection_parameters']['u0']).repeat(nb_frames)
intrinsics[:, 3] = torch.tensor(intrinsics_dict['projection_parameters']['v0']).repeat(nb_frames)


In [106]:
# Camera parameters
poses = torch.cat((cam2world12, two_zeros, intrinsics), dim=1)
poses.shape

image_dict['cameras'] = poses

In [170]:
# Convert images and save them to image_dict

flattened_images = []

for i, filename in enumerate(filenames):
    if i < nb_frames:
        source_path = os.path.join(source_folder, filename)
        image = Image.open(source_path)
        
        # Convert image to bytes
        byte_stream = BytesIO()
        image.save(byte_stream, format='PNG')  # Choose appropriate format, e.g., JPEG, PNG, etc.
        image_bytes = byte_stream.getvalue()
        byte_stream.close()

        # Convert bytes to tensor
        frameTensor = torch.tensor(np.frombuffer(image_bytes, dtype=np.uint8))
        
        flattened_images.append(frameTensor)
        

In [169]:
# Test one image
# byte_stream = BytesIO()
# image.save(byte_stream, format='PNG')  # Choose appropriate format, e.g., JPEG, PNG, etc.
# image_bytes = byte_stream.getvalue()
# byte_stream.close()

# frameTensor = torch.tensor(np.frombuffer(image_bytes, dtype=np.uint8))

# Check if the image is loaded correctly
# image2 = Image.open(BytesIO(frameTensor.numpy().tobytes()))
# image2

In [ ]:
# Save the images
image_dict['images'] = flattened_images

# Save the image_dict to a .torch file
data = [image_dict]

image_dict_name = "000000"
torch.save(data, f'{target_folder}/{image_dict_name}.torch')
torch.save(data, f'/home/angelika/pixelsplat_fisheye/datasets/kitti360/test/{image_dict_name}.torch')


In [179]:
image_dict

{'url': '',
 'key': 'kitti360_cam02_0_99',
 'timestamps': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
         54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
         72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
         90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 'cameras': tensor([[-3.5136e-01, -3.3125e-02,  9.3566e-01,  ...,  1.3358e+03,
           7.1694e+02,  7.0576e+02],
         [-3.5136e-01, -3.3125e-02,  9.3566e-01,  ...,  1.3358e+03,
           7.1694e+02,  7.0576e+02],
         [-3.4567e-01, -3.3660e-02,  9.3775e-01,  ...,  1.3358e+03,
           7.1694e+02,  7.0576e+02],
         ...,
         [ 8.5305e-01,  1.4611e-02,  5.2163e-01,  ...,  1.3358e+03,
           7.1694e+02,  7.0576e+02],
         [ 8.6450e-01,  1.4823e-02,  5.024